<a href="https://colab.research.google.com/github/RogerReateguiUTEC/Tareas_Programacion101/blob/main/Copia_de_Examen_Final_Programacion_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd

In [70]:
# Crear una tabla inicial con datos de ejemplo
data = {
    "Correlativo": [1, 2, 3],
    "DNI": ["12345678A", "87654321B", "11223344C"],
    "Paciente": ["Juan Pérez", "María López", "Carlos Ruiz"],
    "Fecha": ["2024-12-13", "2024-12-14", "2024-12-15"],
    "Hora": ["10:00 AM", "11:00 AM", "9:30 AM"],
    "Doctor": ["Dra. Gómez", "Dr. Martínez", "Dra. Gómez"],
    "Estado": ["Programada", "Programada", "Cancelada"],
}

# Crear un DataFrame
citas = pd.DataFrame(data)

citas["Llave"] = citas["Correlativo"].astype(str) + "-" + citas["DNI"]

citas = citas.set_index("Llave")

citas.head()

,Correlativo,DNI,Paciente,Fecha,Hora,Doctor,Estado
Llave,,,,,,,
1-12345678A,1,12345678A,Juan Pérez,2024-12-13,10:00 AM,Dra. Gómez,Programada
2-87654321B,2,87654321B,María López,2024-12-14,11:00 AM,Dr. Martínez,Programada
3-11223344C,3,11223344C,Carlos Ruiz,2024-12-15,9:30 AM,Dra. Gómez,Cancelada


#GENERAR LAS FUNCIONES

In [37]:
def agregarCita(citas_df, correlativo, dni, paciente, fecha, hora, doctor, estado="Programada"):
    llave = f"{correlativo}-{dni}"

    #verifica si la llave ya existe
    if llave in citas_df.index:
        print(f"Error: Ya existe una cita con la llave {llave}.")
        return citas_df


    # Verificar si el doctor está libre en esa fecha y hora
    doctor_ocupado = citas_df[
        (citas_df["Fecha"] == fecha) &
        (citas_df["Hora"] == hora) &
        (citas_df["Doctor"] == doctor)
    ]

    if not doctor_ocupado.empty:
        print(f"Error: El doctor {doctor} ya tiene una cita programada el {fecha} a las {hora}.")
        return citas_df

    nueva_cita = {
        "Correlativo": correlativo,
        "DNI": dni,
        "Paciente": paciente,
        "Fecha": fecha,
        "Hora": hora,
        "Doctor": doctor,
        "Estado": estado,
    }
    citas_df.loc[llave] = nueva_cita
    print(f"Cita creada con éxito para {paciente}.")
    return citas_df

In [ ]:
def buscarCitaPorPaciente(citas_df, criterio):
    citas_paciente = citas_df[(citas_df["Paciente"] == criterio) | (citas_df["DNI"] == criterio)]

    if citas_paciente.empty:
        print(f"No se encontraron citas para el criterio '{criterio}'.")
    return citas_paciente

In [15]:
def buscarCitaPorFecha(citas_df, fecha):
    citas_en_fecha = citas_df[citas_df["Fecha"] == fecha]
    if citas_en_fecha.empty:
        print(f"No se encontraron citas para la fecha {fecha}.")
    return citas_en_fecha

In [51]:
def listarCitas(citas_df):
    # Convertir la columna 'Fecha' a tipo datetime para asegurarse de que se ordene correctamente
    citas_df["Fecha"] = pd.to_datetime(citas_df["Fecha"])

    # Ordenar las citas por 'Fecha' y luego por 'Hora'
    citas_ordenadas = citas_df.sort_values(by=["Fecha", "Hora"])

    return citas_ordenadas

In [44]:
def modificarCita(citas_df, llave, **nuevos_datos):
    if llave not in citas_df.index:
        print(f"Error: No se encontró una cita con la llave {llave}.")
        return citas_df

    # Verificar si el doctor está libre en esa fecha y hora
    doctor_ocupado = citas_df[
        (citas_df["Fecha"] == nuevos_datos['Fecha']) &
        (citas_df["Hora"] == nuevos_datos['Hora']) &
        (citas_df["Doctor"] == nuevos_datos['Doctor'])
    ]

    if not doctor_ocupado.empty:
        print(f"Error: El doctor {nuevos_datos['Doctor']} ya tiene una cita programada el {nuevos_datos['Fecha']} a las {nuevos_datos['Hora']}.")
        return citas_df

    # Actualizar los datos de la cita
    for columna, valor in nuevos_datos.items():
        if columna in citas_df.columns:
            citas_df.at[llave, columna] = valor
        else:
            print(f"Advertencia: La columna '{columna}' no existe en el DataFrame.")

    print(f"Cita con llave {llave} actualizada con éxito.")
    return citas_df

# ¿Cómo organizarían internamente la información para responder preguntas del
# negocio, como horas pico, citas más frecuentes o razones de cancelación (si
# llegaran a agregarse estos datos en el futuro)?

In [74]:
horas_pico = citas["Hora"].value_counts().sort_values(ascending=False)
horas_pico.head()

,count
Hora,
10:00 AM,1
11:00 AM,1
9:30 AM,1


In [75]:
doctores = citas["Doctor"].value_counts().sort_values(ascending=False)
doctores.head()

,count
Doctor,
Dra. Gómez,2
Dr. Martínez,1


In [77]:
estados = citas["Estado"].value_counts()
estados.head()

,count
Estado,
Programada,2
Cancelada,1


In [79]:
citas_por_dia = citas["Fecha"].value_counts().sort_index()
citas_por_dia.head()

,count
Fecha,
2024-12-13,1
2024-12-14,1
2024-12-15,1


In [81]:
pacientes = citas["Paciente"].value_counts().sort_values(ascending=False)
pacientes.head()

,count
Paciente,
Juan Pérez,1
María López,1
Carlos Ruiz,1


# LLAMADO DE FUNCIONES

PARA LA CREACIÓN DE UNA NUEVA CITA O MODIFICACIÓN DE CITA DEBEMOS:


1. BUSCAR SI EL PACIENTE ESTÁ DISPONIBLE


*   LA BÚSQUEDA LA PODEMOS HACER POR DNI O POR NOMBRE



In [52]:
criterio_buscar = "44833666" #nombre o dni
citas_paciente = buscarCitaPorPaciente(citas, criterio_buscar)

citas_paciente.head()

No se encontraron citas para el criterio '44833666'.


,Correlativo,DNI,Paciente,Fecha,Hora,Doctor,Estado
Llave,,,,,,,


* TAMBIÉN PODEMOS LISTAR TODA LA LISTA DE CITAS

In [54]:
citas_ordenadas = listarCitas(citas)
print("\nCitas ordenadas por fecha y hora:")
citas_ordenadas.head()


Citas ordenadas por fecha y hora:


,Correlativo,DNI,Paciente,Fecha,Hora,Doctor,Estado
Llave,,,,,,,
1-12345678A,1,12345678A,Juan Pérez,2024-12-13,10:00 AM,Dra. Gómez,Programada
1-5555555,1,5555555,Pepito Flores,2024-12-13,10:00 AM,Dra. Gómez,Reprogramada
2-87654321B,2,87654321B,María López,2024-12-14,11:00 AM,Dr. Martínez,Programada
4-33445566D,4,33445566D,Laura Méndez,2024-12-14,11:30 AM,Dr. Martínez,Reprogramada
3-11223344C,3,11223344C,Carlos Ruiz,2024-12-15,9:30 AM,Dra. Gómez,Cancelada


2. SI LA CITA TODAVÍA NO EXISTE, UTILIZAMOS LA FUNCIÓN AGREGARCITA

In [56]:
# Usar la función para crear una nueva cita
citas = agregarCita(citas, 1, "44833666", "Roger", "2024-12-13", "12:00 PM", "Dr. Martínez")

# Mostrar el DataFrame actualizado
print("\nTabla actualizada de citas:")
citas.head(10)

Error: Ya existe una cita con la llave 1-44833666.

Tabla actualizada de citas:


,Correlativo,DNI,Paciente,Fecha,Hora,Doctor,Estado
Llave,,,,,,,
1-12345678A,1,12345678A,Juan Pérez,2024-12-13 00:00:00,10:00 AM,Dra. Gómez,Programada
2-87654321B,2,87654321B,María López,2024-12-14 00:00:00,11:00 AM,Dr. Martínez,Programada
3-11223344C,3,11223344C,Carlos Ruiz,2024-12-15 00:00:00,9:30 AM,Dra. Gómez,Cancelada
4-33445566D,4,33445566D,Laura Méndez,2024-12-14 00:00:00,11:30 AM,Dr. Martínez,Reprogramada
1-5555555,1,5555555,Pepito Flores,2024-12-13 00:00:00,10:00 AM,Dra. Gómez,Reprogramada
1-44833666,1,44833666,Roger,2024-12-13,12:00 PM,Dr. Martínez,Programada


3. SI LA CITA SÍ EXISTE, UTILIZAMOS LA FUNCIÓN MODIFICAR CITA

In [58]:
llave_a_modificar = "1-44833666"  # La llave de la cita que se quiere modificar
nuevos_datos = {
    "Fecha": "2024-12-12",  # Cambiar la fecha
    "Doctor": "Dr. Martínez",
    "Hora": "11:00 AM",     # Cambiar la hora
    "Estado": "Reprogramada"  # Cambiar el estado
}

# Llamar a la función para modificar la cita
citas_actualizadas = modificarCita(citas, llave_a_modificar, **nuevos_datos)

# Mostrar las citas actualizadas
print("\nCitas actualizadas:")
citas_actualizadas.head(10)

Error: El doctor Dr. Martínez ya tiene una cita programada el 2024-12-12 a las 11:00 AM.

Citas actualizadas:


,Correlativo,DNI,Paciente,Fecha,Hora,Doctor,Estado
Llave,,,,,,,
1-12345678A,1,12345678A,Juan Pérez,2024-12-13 00:00:00,10:00 AM,Dra. Gómez,Programada
2-87654321B,2,87654321B,María López,2024-12-14 00:00:00,11:00 AM,Dr. Martínez,Programada
3-11223344C,3,11223344C,Carlos Ruiz,2024-12-15 00:00:00,9:30 AM,Dra. Gómez,Cancelada
4-33445566D,4,33445566D,Laura Méndez,2024-12-14 00:00:00,11:30 AM,Dr. Martínez,Reprogramada
1-5555555,1,5555555,Pepito Flores,2024-12-13 00:00:00,10:00 AM,Dra. Gómez,Reprogramada
1-44833666,1,44833666,Roger,2024-12-12,11:00 AM,Dr. Martínez,Reprogramada
